In [1]:
%cd ..

D:\Projects\AI_projects\safety_helmet


## Import Libraries

In [2]:
import optuna
import yaml
import mlflow
import mlflow.pytorch
from ultralytics import YOLO

D:\Projects\AI_projects\safety_helmet\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read params file

In [3]:
with open(r"params.yaml", "r") as f:
    params = yaml.safe_load(f)

In [4]:
params

{'lr': {'min': 0.01, 'max': 0.1},
 'dp': {'min': 0.1, 'max': 0.6},
 'batch_options': [32, 64]}

In [5]:
params["lr"]["min"]

0.01

In [6]:
min_hp_lr = params["lr"]["min"]
max_hp_lr = params["lr"]["max"]


min_hp_dp = params["dp"]["min"]
max_hp_dp = params["dp"]["max"]

batch_options = params["batch_options"]

In [7]:
def goal_func(trial):
    
    lr = round(trial.suggest_float("lr", min_hp_lr, max_hp_lr),4)
    dropout = round(trial.suggest_float("dropout", min_hp_dp, max_hp_dp),2)
    batch = trial.suggest_categorical("batch", batch_options)
    

    model = YOLO("yolo11n.pt")

    model.train(
        data=r"safety-helmet-1\data.yaml",
        imgsz=128,
        epochs=100,
        project="runs",
        name=f"train_run_{trial.number}",
        
        lr0=lr,
        dropout=dropout,
        batch=batch,
        
        verbose=False
        )
    
    results = model.val(name=f"eval_run_{trial.number}")    
    map50 = results.box.map50
        
    with mlflow.start_run():
        mlflow.log_params({"lr": lr,"dropout": dropout,"batch": batch})
        mlflow.log_metric("map50", map50)
        model.export(format="onnx")
    return map50

mlflow.set_experiment("testing")
study = optuna.create_study(direction="maximize")
study.optimize(goal_func, n_trials=2)

print("Best trial:", study.best_trial)

2025/06/18 16:18:34 INFO mlflow.tracking.fluent: Experiment with name 'testing' does not exist. Creating a new experiment.


[I 2025-06-18 16:18:34,945] A new study created in memory with name: no-name-dd935590-b0d4-4090-b7c2-346c783db869


New https://pypi.org/project/ultralytics/8.3.156 available  Update with 'pip install -U ultralytics'


Ultralytics 8.3.155  Python-3.11.11 torch-2.7.1+cu126 CUDA:0 (Quadro P2000, 4096MiB)


engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=safety-helmet-1\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.17, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=128, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0105, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_run_0, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=runs, rect=False, resume=False, retina_masks=F

...

ONNX: slimming with onnxslim 0.1.57...


ONNX: export success  2.1s, saved as 'runs\train_run_1\weights\best.onnx' (9.9 MB)



Export complete (2.2s)
Results saved to D:\Projects\AI_projects\safety_helmet\runs\train_run_1\weights
Predict:         yolo predict task=detect model=runs\train_run_1\weights\best.onnx imgsz=128  
Validate:        yolo val task=detect model=runs\train_run_1\weights\best.onnx imgsz=128 data=safety-helmet-1\data.yaml  
Visualize:       https://netron.app


[I 2025-06-18 16:32:34,721] Trial 1 finished with value: 0.9564814152230414 and parameters: {'lr': 0.07358603978013765, 'dropout': 0.3204188324611773, 'batch': 32}. Best is trial 1 with value: 0.9564814152230414.


Best trial: FrozenTrial(number=1, state=1, values=[0.9564814152230414], datetime_start=datetime.datetime(2025, 6, 18, 16, 25, 17, 827867), datetime_complete=datetime.datetime(2025, 6, 18, 16, 32, 34, 721295), params={'lr': 0.07358603978013765, 'dropout': 0.3204188324611773, 'batch': 32}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.1, log=False, low=0.01, step=None), 'dropout': FloatDistribution(high=0.6, log=False, low=0.1, step=None), 'batch': CategoricalDistribution(choices=(32, 64))}, trial_id=1, value=None)
